In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import scipy.io.wavfile as wavfile
import scipy.signal

from util.plotting import compute_fft_plot_from_sample_rate
from util.data_io import read_rtl_raw_data, read_gqrx_raw_data
from util.filtering import low_pass_filter_complex_signal, band_pass_filter_complex_signal, low_pass_filter_real_signal
from util.phase_locked_loop import phase_locked_loop

# Test PLL with a dummy signal

## Generate a test signal

In [ ]:
fs = 5000

signal_length = 10
time_vector = np.linspace(0, signal_length, signal_length*fs)
signal_frequency = 225
phase_offset = np.pi/2


# Interestingly, the PLL I've used as reference are just copying positive frequencies. If we use a
# positive frequency, then we get very little error. Using a cos does also work though.

# signal = np.cos(2*np.pi*signal_frequency*time_vector + phase_offset)
signal = np.exp(2j*np.pi*signal_frequency*time_vector + 2j*phase_offset)

## View the test signal

In [ ]:
start = 0
end = int(0.05*fs)

time = time_vector[start:end]
fig = go.Figure()
fig.add_scatter(x=time, y=np.real(signal[start:end]).astype(np.float32), name="original")
fig.show()

In [ ]:
f,m = compute_fft_plot_from_sample_rate(signal, fs)
fig = go.Figure()
fig.add_scatter(x=f,y=m)
fig.show()

## Generate PLL reference and harmonic at double the frequency

In [ ]:
output, output_harmonic, error = phase_locked_loop(signal, fs, 215, 20, output_frequency_harmonic=2)

## View the results from the PLL and compare to the input

In [ ]:
f,m = compute_fft_plot_from_sample_rate(output, fs)

fig = go.Figure()
fig.add_scatter(x=f,y=m)
fig.show()

In [ ]:
f,m = compute_fft_plot_from_sample_rate(output_harmonic, fs)

fig = go.Figure()
fig.add_scatter(x=f,y=m)
fig.show()

In [ ]:
start = int(0*fs)
end = int(1.0*fs)

time = time_vector[start:end]
fig = go.Figure()
fig.add_scatter(x=time, y=np.real(signal[start:end]).astype(np.float32), name="original")
fig.add_scatter(x=time, y=np.real(output[start:end]).astype(np.float32), name="pll")
fig.add_scatter(x=time, y=np.real(output_harmonic[start:end]).astype(np.float32), name="pll harmonic")
fig.show()